## Weather Cleaning Notebook ##


Schoeggl_Vorauer 2020

cleaning weather data

datasetlink: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

In [122]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter 
from sqlalchemy import create_engine
import swifter # swifter is for multiprocessing pandas apply functions
from Function_Definitions import *
from helferlein import implottop = df['WindGustDir'].describe().top
df['WindGustDir'].fillna(top, inplace=True)
df.isnull().sum()

In [123]:
# import weatherdataset as csv
weather = pd.read_csv(r'..\Data\Raw\weatherAUS.csv')

In [124]:
#keyelements(columnnames output)
weather.keys()

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [125]:
#weather location output
weather['Location']

0         Albury
1         Albury
2         Albury
3         Albury
4         Albury
           ...  
142188     Uluru
142189     Uluru
142190     Uluru
142191     Uluru
142192     Uluru
Name: Location, Length: 142193, dtype: object

In [126]:
#looking for unique locations
weatherloc = weather.Location.unique()

In [127]:
#weather column Locations asign to new variable
weatherloc = pd.DataFrame(weatherloc, columns=['Location']) 

**split appended names** **change seperator and join saperated names**

In [128]:
#call stringchange function from Function_Definitions file
weather = stringchange(weather)
weather = weather.replace({'Pearce R A A F': 'Pearce'})

In [129]:
#weather column assign Locations to new variable 
weatherloc = weather.Location.unique()
weatherloc = pd.DataFrame(weatherloc, columns=['Location']) 

In [130]:
#assign unique locations nach weatherloc
np.all(weather.Location.unique() == weatherloc.Location.unique())

True

**georeferencing locations**

In [131]:
#applying geocoordinates to locations
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
gc = Nominatim(user_agent="fintu-blog-geocoding-python")
#short test
bla1 , bla2 = gc.geocode("Pears, Australia")[1]

In [132]:
#applying geocoordinates to locations / define ratelimiter
def rate_limited_geocode(query):
    geocode = RateLimiter(gc.geocode, min_delay_seconds=1)
    return geocode(query)
#geocode here is another variable than above
def geocodeb(row):
    lookup_query = row["Location"] + ", " + "Australia"
    lookup_result = rate_limited_geocode(lookup_query)
    return lookup_result
weatherloc["loctemp"] = weatherloc.apply(geocodeb, axis=1)

In [133]:
# making sure that it finds values to all locations
weatherloc.isna().sum()

Location    0
loctemp     0
dtype: int64

In [134]:
#loctemp contains all geo-informations given above. here we extract longitude and latitude information.
tempbla = [i[1] for i in weatherloc.loctemp]
weatherloc['latitude'] , weatherloc['longitude'] = np.array(tempbla).T
weatherloc = weatherloc.drop(['loctemp'],1)

**assign the coordiantes given from above function to locations in weather**

In [135]:
#filling function for longitude, latitude into correct rows
def reverse_filling(df1, df2):
    templatitude = np.zeros(df1.shape[0])
    templongitude = np.zeros(df1.shape[0])
    for i in df1.Location.unique():
        templatitude[df1.Location == i] = df2[df2.Location==i].latitude
        templongitude[df1.Location == i] = df2[df2.Location==i].longitude
    df1['latitude'] = templatitude
    df1['longitude'] = templongitude
    return df1

In [136]:
#mapping of latitude and longitude values in weatherloc to the corresponding locations in weather
weather = reverse_filling(weather, weatherloc)

In [137]:
#short test
weather.latitude

0        -36.080477
1        -36.080477
2        -36.080477
3        -36.080477
4        -36.080477
            ...    
142188   -25.345554
142189   -25.345554
142190   -25.345554
142191   -25.345554
142192   -25.345554
Name: latitude, Length: 142193, dtype: float64

In [138]:
weather.shape

(142193, 26)

In [139]:
weather = weather.rename(columns={'Date': 'acq_date'})

In [140]:
#set aqc_date to datetime
weather['acq_date'] = pd.to_datetime(weather['acq_date'])
weather.head()

,acq_date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,latitude,longitude
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,1007.1,8.0,NaN,16.9,21.8,No,0.0,No,-36.080477,146.91628
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,1007.8,NaN,NaN,17.2,24.3,No,0.0,No,-36.080477,146.91628
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,1008.7,NaN,2.0,21.0,23.2,No,0.0,No,-36.080477,146.91628
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1012.8,NaN,NaN,18.1,26.5,No,1.0,No,-36.080477,146.91628
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,1006.0,7.0,8.0,17.8,29.7,No,0.2,No,-36.080477,146.91628


In [141]:
weather.Evaporation.isna().sum()

60843

In [142]:
weather.isna().sum()

acq_date             0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RISK_MM              0
RainTomorrow         0
latitude             0
longitude            0
dtype: int64

In [143]:
weather.head()

,acq_date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,latitude,longitude
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,1007.1,8.0,NaN,16.9,21.8,No,0.0,No,-36.080477,146.91628
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,1007.8,NaN,NaN,17.2,24.3,No,0.0,No,-36.080477,146.91628
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,1008.7,NaN,2.0,21.0,23.2,No,0.0,No,-36.080477,146.91628
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1012.8,NaN,NaN,18.1,26.5,No,1.0,No,-36.080477,146.91628
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,1006.0,7.0,8.0,17.8,29.7,No,0.2,No,-36.080477,146.91628


In [144]:
weather = year_month(weather)

In [145]:
weather.Evaporation.max()

145.0

In [146]:
weather = weather.drop(['Sunshine'], 1)

In [147]:
weather = weather.drop(['RISK_MM'], 1)

In [148]:
rt_mode = weather['RainToday'].describe().top


In [149]:
weather['RainToday'].fillna(rt_mode, inplace=True)

In [150]:
weather

,acq_date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,latitude,longitude,Year,Month
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,W,44.0,W,WNW,...,8.0,NaN,16.9,21.8,No,No,-36.080477,146.916280,2008,12
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,WNW,44.0,NNW,WSW,...,NaN,NaN,17.2,24.3,No,No,-36.080477,146.916280,2008,12
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,WSW,46.0,W,WSW,...,NaN,2.0,21.0,23.2,No,No,-36.080477,146.916280,2008,12
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NE,24.0,SE,E,...,NaN,NaN,18.1,26.5,No,No,-36.080477,146.916280,2008,12
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,W,41.0,ENE,NW,...,7.0,8.0,17.8,29.7,No,No,-36.080477,146.916280,2008,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,NaN,E,31.0,ESE,E,...,NaN,NaN,9.4,20.9,No,No,-25.345554,131.036961,2017,6
142189,2017-06-21,Uluru,2.8,23.4,0.0,NaN,E,31.0,SE,ENE,...,NaN,NaN,10.1,22.4,No,No,-25.345554,131.036961,2017,6
142190,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NNW,22.0,SE,N,...,NaN,NaN,10.9,24.5,No,No,-25.345554,131.036961,2017,6
142191,2017-06-23,Uluru,5.4,26.9,0.0,NaN,N,37.0,SE,WNW,...,NaN,NaN,12.5,26.1,No,No,-25.345554,131.036961,2017,6


In [153]:
top = weather['WindGustDir'].describe().top
weather['WindGustDir'].fillna(top, inplace=True)
weather.isnull().sum()

acq_date             0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
WindGustDir          0
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday            0
RainTomorrow         0
latitude             0
longitude            0
Year                 0
Month                0
dtype: int64

In [154]:
for col in weather.columns:
    print(col + ' has ' + str(round((weather[col].isnull().sum() / weather.shape[0]) * 100, 2)) + '% missing values')

acq_date has 0.0% missing values
Location has 0.0% missing values
MinTemp has 0.45% missing values
MaxTemp has 0.23% missing values
Rainfall has 0.99% missing values
Evaporation has 42.79% missing values
WindGustDir has 0.0% missing values
WindGustSpeed has 6.52% missing values
WindDir9am has 7.04% missing values
WindDir3pm has 2.66% missing values
WindSpeed9am has 0.95% missing values
WindSpeed3pm has 1.85% missing values
Humidity9am has 1.25% missing values
Humidity3pm has 2.54% missing values
Pressure9am has 9.86% missing values
Pressure3pm has 9.83% missing values
Cloud9am has 37.74% missing values
Cloud3pm has 40.15% missing values
Temp9am has 0.64% missing values
Temp3pm has 1.92% missing values
RainToday has 0.0% missing values
RainTomorrow has 0.0% missing values
latitude has 0.0% missing values
longitude has 0.0% missing values
Year has 0.0% missing values
Month has 0.0% missing values


In [155]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   acq_date       142193 non-null  datetime64[ns]
 1   Location       142193 non-null  object        
 2   MinTemp        141556 non-null  float64       
 3   MaxTemp        141871 non-null  float64       
 4   Rainfall       140787 non-null  float64       
 5   Evaporation    81350 non-null   float64       
 6   WindGustDir    142193 non-null  object        
 7   WindGustSpeed  132923 non-null  float64       
 8   WindDir9am     132180 non-null  object        
 9   WindDir3pm     138415 non-null  object        
 10  WindSpeed9am   140845 non-null  float64       
 11  WindSpeed3pm   139563 non-null  float64       
 12  Humidity9am    140419 non-null  float64       
 13  Humidity3pm    138583 non-null  float64       
 14  Pressure9am    128179 non-null  float64       
 15  

In [156]:
weather.head()

,acq_date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,latitude,longitude,Year,Month
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,W,44.0,W,WNW,...,8.0,NaN,16.9,21.8,No,No,-36.080477,146.91628,2008,12
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,WNW,44.0,NNW,WSW,...,NaN,NaN,17.2,24.3,No,No,-36.080477,146.91628,2008,12
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,WSW,46.0,W,WSW,...,NaN,2.0,21.0,23.2,No,No,-36.080477,146.91628,2008,12
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NE,24.0,SE,E,...,NaN,NaN,18.1,26.5,No,No,-36.080477,146.91628,2008,12
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,W,41.0,ENE,NW,...,7.0,8.0,17.8,29.7,No,No,-36.080477,146.91628,2008,12


In [157]:
weather['Rainfall'].fillna(0, inplace=True)

In [158]:
weather.isnull().sum()

acq_date             0
Location             0
MinTemp            637
MaxTemp            322
Rainfall             0
Evaporation      60843
WindGustDir          0
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday            0
RainTomorrow         0
latitude             0
longitude            0
Year                 0
Month                0
dtype: int64

In [159]:
rt_mode = weather['RainToday'].describe().top
weather['RainToday'].fillna(rt_mode, inplace=True)

In [160]:
weather['Cloud9am'].fillna(0, inplace=True)
weather['Cloud3pm'].fillna(0, inplace=True)

In [161]:
weather['Evaporation'].fillna(0, inplace=True)

In [162]:
weather.isna().sum()

acq_date             0
Location             0
MinTemp            637
MaxTemp            322
Rainfall             0
Evaporation          0
WindGustDir          0
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am             0
Cloud3pm             0
Temp9am            904
Temp3pm           2726
RainToday            0
RainTomorrow         0
latitude             0
longitude            0
Year                 0
Month                0
dtype: int64

In [163]:
# for whole dataframe
#weather = weather.replace(np.nan, 0)
weather

,acq_date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,latitude,longitude,Year,Month
0,2008-12-01,Albury,13.4,22.9,0.6,0.0,W,44.0,W,WNW,...,8.0,0.0,16.9,21.8,No,No,-36.080477,146.916280,2008,12
1,2008-12-02,Albury,7.4,25.1,0.0,0.0,WNW,44.0,NNW,WSW,...,0.0,0.0,17.2,24.3,No,No,-36.080477,146.916280,2008,12
2,2008-12-03,Albury,12.9,25.7,0.0,0.0,WSW,46.0,W,WSW,...,0.0,2.0,21.0,23.2,No,No,-36.080477,146.916280,2008,12
3,2008-12-04,Albury,9.2,28.0,0.0,0.0,NE,24.0,SE,E,...,0.0,0.0,18.1,26.5,No,No,-36.080477,146.916280,2008,12
4,2008-12-05,Albury,17.5,32.3,1.0,0.0,W,41.0,ENE,NW,...,7.0,8.0,17.8,29.7,No,No,-36.080477,146.916280,2008,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,0.0,E,31.0,ESE,E,...,0.0,0.0,9.4,20.9,No,No,-25.345554,131.036961,2017,6
142189,2017-06-21,Uluru,2.8,23.4,0.0,0.0,E,31.0,SE,ENE,...,0.0,0.0,10.1,22.4,No,No,-25.345554,131.036961,2017,6
142190,2017-06-22,Uluru,3.6,25.3,0.0,0.0,NNW,22.0,SE,N,...,0.0,0.0,10.9,24.5,No,No,-25.345554,131.036961,2017,6
142191,2017-06-23,Uluru,5.4,26.9,0.0,0.0,N,37.0,SE,WNW,...,0.0,0.0,12.5,26.1,No,No,-25.345554,131.036961,2017,6


In [166]:
weather.keys()

Index(['acq_date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow', 'latitude', 'longitude', 'Year',
       'Month'],
      dtype='object')

In [167]:
weather[['Humidity3pm','Temp3pm','Temp9am','MinTemp', 'MaxTemp']] = weather[['Humidity3pm','Temp3pm','Temp9am','MinTemp', 'MaxTemp']].fillna(0).astype(float)

In [168]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   acq_date       142193 non-null  datetime64[ns]
 1   Location       142193 non-null  object        
 2   MinTemp        142193 non-null  float64       
 3   MaxTemp        142193 non-null  float64       
 4   Rainfall       142193 non-null  float64       
 5   Evaporation    142193 non-null  float64       
 6   WindGustDir    142193 non-null  object        
 7   WindGustSpeed  132923 non-null  float64       
 8   WindDir9am     132180 non-null  object        
 9   WindDir3pm     138415 non-null  object        
 10  WindSpeed9am   140845 non-null  float64       
 11  WindSpeed3pm   139563 non-null  float64       
 12  Humidity9am    140419 non-null  float64       
 13  Humidity3pm    142193 non-null  float64       
 14  Pressure9am    128179 non-null  float64       
 15  

In [169]:
weather[['WindGustSpeed',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm']] = weather[['WindGustSpeed',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm']].fillna(0).astype(float)

In [170]:
weather[['WindGustDir', 'WindDir9am', 'WindDir3pm']] = weather[['WindGustDir', 'WindDir9am', 'WindDir3pm']].fillna(0)

In [171]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   acq_date       142193 non-null  datetime64[ns]
 1   Location       142193 non-null  object        
 2   MinTemp        142193 non-null  float64       
 3   MaxTemp        142193 non-null  float64       
 4   Rainfall       142193 non-null  float64       
 5   Evaporation    142193 non-null  float64       
 6   WindGustDir    142193 non-null  object        
 7   WindGustSpeed  142193 non-null  float64       
 8   WindDir9am     142193 non-null  object        
 9   WindDir3pm     142193 non-null  object        
 10  WindSpeed9am   142193 non-null  float64       
 11  WindSpeed3pm   142193 non-null  float64       
 12  Humidity9am    142193 non-null  float64       
 13  Humidity3pm    142193 non-null  float64       
 14  Pressure9am    142193 non-null  float64       
 15  

In [174]:
weather= get_state(weather)

In [175]:
weather.acq_date.max()
weather.keys()

Index(['acq_date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow', 'latitude', 'longitude', 'Year',
       'Month', 'State'],
      dtype='object')

In [176]:
weather.WindGustDir.unique()

array(['W', 'WNW', 'WSW', 'NE', 'NNW', 'N', 'NNE', 'SW', 'ENE', 'SSE',
       'S', 'NW', 'SE', 'ESE', 'E', 'SSW'], dtype=object)

In [177]:
weather = weather.replace(to_replace = weather.WindGustDir.unique()[[12,13,10,2,15,0,1,11,3,7,8,14,6,5,4,9]], value =[360/16*i for i in range(16)])
weather.WindGustDir.unique()[[12,13,10,2,15,0,1,11,3,7,8,14,6,5,4,9]]

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ,
       202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5])

In [178]:
weather = weather.replace(to_replace = ['No', 'Yes'], value =[False , True])

In [179]:
weather.RainTomorrow.unique()

array([False,  True])

In [181]:
weather

,acq_date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,latitude,longitude,Year,Month,State
0,2008-12-01,Albury,13.4,22.9,0.6,0.0,112.5,44.0,112.5,135.0,...,0.0,16.9,21.8,False,False,-36.080477,146.916280,2008,12,VIC
1,2008-12-02,Albury,7.4,25.1,0.0,0.0,135.0,44.0,315.0,67.5,...,0.0,17.2,24.3,False,False,-36.080477,146.916280,2008,12,VIC
2,2008-12-03,Albury,12.9,25.7,0.0,0.0,67.5,46.0,112.5,67.5,...,2.0,21.0,23.2,False,False,-36.080477,146.916280,2008,12,VIC
3,2008-12-04,Albury,9.2,28.0,0.0,0.0,180.0,24.0,0.0,247.5,...,0.0,18.1,26.5,False,False,-36.080477,146.916280,2008,12,VIC
4,2008-12-05,Albury,17.5,32.3,1.0,0.0,112.5,41.0,225.0,157.5,...,8.0,17.8,29.7,False,False,-36.080477,146.916280,2008,12,VIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,0.0,247.5,31.0,22.5,247.5,...,0.0,9.4,20.9,False,False,-25.345554,131.036961,2017,6,NT
142189,2017-06-21,Uluru,2.8,23.4,0.0,0.0,247.5,31.0,0.0,225.0,...,0.0,10.1,22.4,False,False,-25.345554,131.036961,2017,6,NT
142190,2017-06-22,Uluru,3.6,25.3,0.0,0.0,315.0,22.0,0.0,292.5,...,0.0,10.9,24.5,False,False,-25.345554,131.036961,2017,6,NT
142191,2017-06-23,Uluru,5.4,26.9,0.0,0.0,292.5,37.0,0.0,135.0,...,0.0,12.5,26.1,False,False,-25.345554,131.036961,2017,6,NT


In [182]:
weather['season'] = (weather.Month%12 + 3)//3

In [183]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 28 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   acq_date       142193 non-null  datetime64[ns]
 1   Location       142193 non-null  object        
 2   MinTemp        142193 non-null  float64       
 3   MaxTemp        142193 non-null  float64       
 4   Rainfall       142193 non-null  float64       
 5   Evaporation    142193 non-null  float64       
 6   WindGustDir    142193 non-null  float64       
 7   WindGustSpeed  142193 non-null  float64       
 8   WindDir9am     142193 non-null  float64       
 9   WindDir3pm     142193 non-null  float64       
 10  WindSpeed9am   142193 non-null  float64       
 11  WindSpeed3pm   142193 non-null  float64       
 12  Humidity9am    142193 non-null  float64       
 13  Humidity3pm    142193 non-null  float64       
 14  Pressure9am    142193 non-null  float64       
 15  

In [184]:
weather

,acq_date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Temp9am,Temp3pm,RainToday,RainTomorrow,latitude,longitude,Year,Month,State,season
0,2008-12-01,Albury,13.4,22.9,0.6,0.0,112.5,44.0,112.5,135.0,...,16.9,21.8,False,False,-36.080477,146.916280,2008,12,VIC,1
1,2008-12-02,Albury,7.4,25.1,0.0,0.0,135.0,44.0,315.0,67.5,...,17.2,24.3,False,False,-36.080477,146.916280,2008,12,VIC,1
2,2008-12-03,Albury,12.9,25.7,0.0,0.0,67.5,46.0,112.5,67.5,...,21.0,23.2,False,False,-36.080477,146.916280,2008,12,VIC,1
3,2008-12-04,Albury,9.2,28.0,0.0,0.0,180.0,24.0,0.0,247.5,...,18.1,26.5,False,False,-36.080477,146.916280,2008,12,VIC,1
4,2008-12-05,Albury,17.5,32.3,1.0,0.0,112.5,41.0,225.0,157.5,...,17.8,29.7,False,False,-36.080477,146.916280,2008,12,VIC,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,0.0,247.5,31.0,22.5,247.5,...,9.4,20.9,False,False,-25.345554,131.036961,2017,6,NT,3
142189,2017-06-21,Uluru,2.8,23.4,0.0,0.0,247.5,31.0,0.0,225.0,...,10.1,22.4,False,False,-25.345554,131.036961,2017,6,NT,3
142190,2017-06-22,Uluru,3.6,25.3,0.0,0.0,315.0,22.0,0.0,292.5,...,10.9,24.5,False,False,-25.345554,131.036961,2017,6,NT,3
142191,2017-06-23,Uluru,5.4,26.9,0.0,0.0,292.5,37.0,0.0,135.0,...,12.5,26.1,False,False,-25.345554,131.036961,2017,6,NT,3


In [185]:
weather.to_csv(r'..\Data\Clean\cweather.csv', index=False)